In [26]:

import json
import requests
image='golang'
registry_url='https://registry-1.docker.io'
auth_url='https://auth.docker.io'
svc_url='registry.docker.io'

def auth_token():
    url = f"{auth_url}/token?service={svc_url}&scope=repository:library/{image}:pull"
    r = requests.get(url)
    return r.json()['token']
TOKEN=auth_token()



In [27]:

def manifest(token, image, digest='latest'):
    headers = {
        'Authorization': f"Bearer {token}",
        'Accept': 'application/vnd.docker.distribution.manifest.list.v2+json',
        'Accept': 'application/vnd.docker.distribution.manifest.v1+json',
        'Accept': 'application/vnd.docker.distribution.manifest.v2+json',
    }
    r = requests.get(f"{registry_url}/v2/library/{image}/manifests/{digest}", headers=headers)
    return r.json()
manifest(TOKEN, image, 'latest')


{'schemaVersion': 2,
 'mediaType': 'application/vnd.docker.distribution.manifest.v2+json',
 'config': {'mediaType': 'application/vnd.docker.container.image.v1+json',
  'size': 7223,
  'digest': 'sha256:015e6b7f599b15be5eecad68608583d19d9fa2c8cc27cb6d254204e080dae199'},
 'layers': [{'mediaType': 'application/vnd.docker.image.rootfs.diff.tar.gzip',
   'size': 49582224,
   'digest': 'sha256:0a9573503463fd3166b5b1f34a7720dac28609e98d731e50e7068f92e79b8545'},
  {'mediaType': 'application/vnd.docker.image.rootfs.diff.tar.gzip',
   'size': 24051259,
   'digest': 'sha256:1ccc26d841b4acc2562483bf393ce1cf8bc358ced09ccd825425226827c79c92'},
  {'mediaType': 'application/vnd.docker.image.rootfs.diff.tar.gzip',
   'size': 64130287,
   'digest': 'sha256:800d84653581fc119cd75cd572fa190d3b813d49221b9e5ee463e3560e2cb342'},
  {'mediaType': 'application/vnd.docker.image.rootfs.diff.tar.gzip',
   'size': 92327514,
   'digest': 'sha256:9211c993294398915ebd0aaf372f7cdb2a3ef81bcbb5af6f71967523546b086d'},
  {'

In [28]:

def blob(token, image, digest, file):
    headers = {
        'Authorization': f"Bearer {token}",
    }
    r = requests.get(f"{registry_url}/v2/library/{image}/blobs/{digest}", headers=headers)
    # do this in parralel
    with open(file, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            f.write(chunk)

def layers(manifest):
    return [l['digest'] for l in manifest['layers']]
token=auth_token()

mf=manifest(token, image, 'latest')
print(layers(mf))
for i, l in enumerate(layers(mf)):
    blob(token, image, l, f"layer{i}.tar.gz")


['sha256:0a9573503463fd3166b5b1f34a7720dac28609e98d731e50e7068f92e79b8545', 'sha256:1ccc26d841b4acc2562483bf393ce1cf8bc358ced09ccd825425226827c79c92', 'sha256:800d84653581fc119cd75cd572fa190d3b813d49221b9e5ee463e3560e2cb342', 'sha256:9211c993294398915ebd0aaf372f7cdb2a3ef81bcbb5af6f71967523546b086d', 'sha256:b05d0f037378c842afdaeba0cb038a42a0a9b4e29367f1b2537ef5388ed13719', 'sha256:9f49ffca687ac5513fd41991a1d5783020961687b147df85d0f43e3c9ff7a084']


In [29]:
!echo "{mf}"


{'schemaVersion': 2, 'mediaType': 'application/vnd.docker.distribution.manifest.v2+json', 'config': {'mediaType': 'application/vnd.docker.container.image.v1+json', 'size': 7223, 'digest': 'sha256:015e6b7f599b15be5eecad68608583d19d9fa2c8cc27cb6d254204e080dae199'}, 'layers': [{'mediaType': 'application/vnd.docker.image.rootfs.diff.tar.gzip', 'size': 49582224, 'digest': 'sha256:0a9573503463fd3166b5b1f34a7720dac28609e98d731e50e7068f92e79b8545'}, {'mediaType': 'application/vnd.docker.image.rootfs.diff.tar.gzip', 'size': 24051259, 'digest': 'sha256:1ccc26d841b4acc2562483bf393ce1cf8bc358ced09ccd825425226827c79c92'}, {'mediaType': 'application/vnd.docker.image.rootfs.diff.tar.gzip', 'size': 64130287, 'digest': 'sha256:800d84653581fc119cd75cd572fa190d3b813d49221b9e5ee463e3560e2cb342'}, {'mediaType': 'application/vnd.docker.image.rootfs.diff.tar.gzip', 'size': 92327514, 'digest': 'sha256:9211c993294398915ebd0aaf372f7cdb2a3ef81bcbb5af6f71967523546b086d'}, {'mediaType': 'application/vnd.docker.ima